In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/NIST_Utils-master/pymetamap")
sys.path
#/home/tay/Documents/DSA-NIST

['',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python27.zip',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/plat-linux2',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-tk',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-old',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-dynload',
 '/home/tay/.local/lib/python2.7/site-packages',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages/IPython/extensions',
 '/home/tay/.ipython',
 '/home/tay/Documents/DSA-NIST/NIST_Utils-master/pymetamap']

In [2]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from NLPutils import NLPutils as NLP
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.snowball import EnglishStemmer # load the stemmer module from NLTK
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer
import pickle
import math
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import time
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import py_trees
import pymetamap #**
import behaviours as be #** be sure to properly map pymetampap -> try in VM
    ### make sure to include behaviours_m.py instead of behaviors.py
from py_trees.blackboard import Blackboard
from scipy import spatial
from pandas import DataFrame
import re
import matplotlib.pyplot as plt
%matplotlib notebook

IndentationError: unexpected indent (ConceptExtract.py, line 271)

In [ ]:
import sys
sys.executable

In [ ]:
sys.path

In [ ]:
class RAAdata(object):
    def __init__(self,text,vital,inter):
        self.text = text
        self.vital = vital
        self.inter = inter
        
def load_RAA_data(path, cv = True): 
    df = pd.read_excel(path)
    interset = set()
    interdict = dict()
    narratives = df['Narrative']
    narratives = [i for i in narratives]
    inters = df['Interventions']
    vitals = df['Vitals']
    vitals = [i for i in vitals]
    interventions = []
    for item in inters:
        inter = item.strip('{}').split('}{')
        inter = [i.split(':')[-1].strip().lower() for i in inter]
        c_int = []
        for j in inter:
            interset.add(j)
            if j in interdict:
                interdict[j] += 1
            else:
                interdict[j] = 1
            c_int.append(j)
        interventions.append(c_int)
    for inter in list(interdict):
        if cv and interdict[inter] < 20: del interdict[inter]
    data = [RAAdata(item,vitals[idx],interventions[idx]) for idx,item in enumerate(narratives)]
    
    return data,interdict

def fullmatch(regex, string, flags=0):
    """Emulate python-3.4 re.fullmatch()."""
    return re.match("(?:" + regex + r")\Z", string, flags=flags)

# preprocess utils
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext

def cleanPunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

def weighted_precision_recall_f1_util (y_test, y_pre, weight = None):
    tp, fp, fn = [0. for _ in range(len(y_pre[0]))], [0. for _ in range(len(y_pre[0]))], \
    [0. for _ in range(len(y_pre[0]))]
    for idx in range(len(y_pre)):
        for i in range(len(y_pre[idx])):
            if y_pre[idx][i] == 1 and y_test[idx][i] == 1: tp[i] += 1
            elif y_pre[idx][i] == 1 and y_test[idx][i] == 0: fp[i] += 1
            elif y_pre[idx][i] == 0 and y_test[idx][i] == 1: fn[i] += 1
    precision = [tp[i] / (tp[i] + fp[i]) if tp[i] > 0 or fp[i] > 0 else 0. for i in range(len(tp))]
    recall = [tp[i] / (tp[i] + fn[i]) if tp[i] > 0 or fn[i] > 0 else 0. for i in range(len(tp))]
    f1 = [2 * precision[i] * recall[i] / (precision[i] + recall[i]) \
         if precision[i] > 0 or recall[i] > 0 else 0. for i in range(len(tp))]
    return np.average(precision, weights = weight), np.average(recall, weights = weight), \
np.average(f1, weights = weight)

def weighted_precision (y_test, y_pre, weight = None):
    precision, _, _ = weighted_precision_recall_f1_util (y_test, y_pre, weight)
    return precision

def weighted_recall (y_test, y_pre, weight = None):
    _, recall, _ = weighted_precision_recall_f1_util (y_test, y_pre, weight)
    return recall

def weighted_f1 (y_test, y_pre, weight = None):
    _, _, f1 = weighted_precision_recall_f1_util (y_test, y_pre, weight)
    return f1

def show_results(scores):
    metrics = ['test_precision_weighted','test_recall_weighted', 'test_f1_weighted',\
            'test_precision_micro', 'test_recall_micro', 'test_f1_micro']
    for metric in metrics:
        print metric + ':' + '%.2f' % np.average(scores[metric])
        
def risk_factor(gt, probs, preds):
    risk = []
    for idx,case in enumerate(probs):
        r = 0
        for i,prob in enumerate(case):
            if preds[idx][i] == 1 and gt[idx][i] == 0:
                r += prob * int2fp_score[num2int[i]] / sum(gt[idx])
            if preds[idx][i] == 0 and gt[idx][i] == 1:
                r += prob * int2fn_score[num2int[i]] / sum(gt[idx])
        risk.append(r)
    return sum(risk) / len(risk)

def trans_prob(probs):
    transed_prob = [[0.] * len(probs) for _ in range(len(probs[0]))]
    for idx, res in enumerate(probs):
        for i, p in enumerate(res):
            if len(p) < 2: transed_prob[i][idx] = 1. - p[0]
            else: transed_prob[i][idx] = p[1]
                
    return transed_prob

def show_test_results(gt, res, prob, class_weight):
    print "precision_micro" + ':' + '%.2f' % precision_score(gt, res, average = 'micro')
    print "recall_micro" + ':' + '%.2f' % recall_score(gt, res, average = 'micro')
    print "f1_micro" + ':' + '%.2f' % f1_score(gt, res, average = 'micro')
    print "precision_weighted" + ':' + '%.2f' % weighted_precision(gt, res, class_weight)
    print "recall_weighted" + ':' + '%.2f' % weighted_recall(gt, res, class_weight)
    print "f1_weighted" + ':' + '%.2f' % weighted_f1(gt, res, class_weight)
    print "risk_factor" + ':' + '%.4f' % risk_factor(gt, prob, res)
    
def filtering(res, prob, threshold):
    for idx, case in enumerate(res):
        for i in range(len(case)):
            if prob[idx][i] < threshold:
                res[idx][i] = 0
                prob[idx][i] = 0.
    return res, prob

In [ ]:
# read labeled cases
docu = './RAA_train.xlsx'
df = pd.read_excel(docu)
vitals = df['Vitals']
train_narratives = df['Narrative']

In [ ]:
#* Declare RAA data object 
#then call your object functions corectly for vitals

In [ ]:
# ##SKIP##
# from tqdm import tqdm_notebook as tqdm
# EKGset = set()
# train_vec = []
# EKGdic = {
#      '':'',
#      'AV_Block_1st_Deg':'AV_Block-1st_Degree',
#      'AV_Block_1st_Degree':'AV_Block-1st_Degree',
#      'AV_Block_2nd_Degree_Type_1':'AV_Block_2nd_Degree_Type_1',
#      'AV_Block_2nd_Degree_Type_2':'AV_Block_2nd_Degree_Type_2',
#      'AV_Block_3rd_Degree':'AV_Block_3rd_Degree',
#      'Asystole':'Asystole',
#      'Artifact':'Artifact',
#      'Atrial_Fibrill':'Atrial_Fibrillation',
#      'Atrial_Fibrillation':'Atrial_Fibrillation',
#      'Atrial_Flutter':'Atrial_Flutter',
#      'Agonal/Idioventricular':'Agonal/Idioventricular',
#      'Juncti':'Junctional_Rhythm',
#      'Junctiona':'Junctional_Rhythm',
#      'Junctional':'Junctional_Rhythm',
#      'Non_STEMI_Anterior_Ischemia':'Non_STEMI_Anterior_Ischemia',
#      'Non_STEMI_Lateral_Ischemia':'Non_STEMI_Lateral_Ischemia',
#      'Other_(Not_Listed)':'Other_(Not_Listed)',
#      'P': 'Paced_Rhythm',
#      'PEA':'Pulseless_Electrical_Activity',
#      'Pac':'Paced_Rhythm',
#      'Paced':'Paced_Rhythm',
#      'Paced_Rhythm':'Paced_Rhythm',
#      'Premature_Ventricular_Contractions':'Premature_Ventricular_Contractions',
#      'Premature_Atrial_Contractions':'Premature_Atrial_Contractions',
#      'Right_Bundle_Branch_Block':'Right_Bundle_Branch_Block',
#      'Left_Bundle_Branch_Block':'Left_Bundle_Branch_Block',
#      'STEMI_Anterior_Ischemia':'STEMI_Anterior_Ischemia',
#      'STEMI_Lateral_Ischemia':'STEMI_Lateral_Ischemia',
#      'STEMI_Inferior_Ischemia':'STEMI_Inferior_Ischemia',
#      'S':'Sinus_Rhythm',
#      'Si':'Sinus_Rhythm',
#      'Sin':'Sinus_Rhythm',
#      'Sinu':'Sinus_Rhythm',
#      'Sinus':'Sinus_Rhythm',
#      'Sinus_':'Sinus_Rhythm',
#      'Sinus_Arrhythmia':'Sinus_Arrhythmia',
#      'Sinus_Bradycardia':'Sinus_Bradycardia',
#      'Sinus_R':'Sinus_Rhythm',
#      'Sinus_Rh':'Sinus_Rhythm',
#      'Sinus_Rhy':'Sinus_Rhythm',
#      'Sinus_Rhyt':'Sinus_Rhythm',
#      'Sinus_Rhyth':'Sinus_Rhythm',
#      'Sinus_Rhythm':'Sinus_Rhythm',
#      'Sinus_Rhythm,Sinus_Tachycardia':'Sinus_Rhythm,Sinus_Tachycardia',
#      'Sinus_T':'Sinus_Tachycardia',
#      'Sinus_Tach':'Sinus_Tachycardia',
#      'Sinus_Tachyc':'Sinus_Tachycardia',
#      'Sinus_Tachycardi':'Sinus_Tachycardia',
#      'Sinus_Tachycardia':'Sinus_Tachycardia',
#      'Supravent':'Supraventricular_Tachycardia',
#      'Supraventricular_Tachycardia':'Supraventricular_Tachycardia',
#      'Torsades_De_Points':'Torsades_De_Points',
#      'Ventricular_Fibrillation':'Ventricular_Fibrillation',
#      'Ventricular_Tachycardia_(With_Pulse)':'Ventricular_Tachycardia',
#      'Ventricular_Tachycardia_(Pulseless)':'Ventricular_Tachycardia_(Pulseless)',
#      'Unknown_AED_Non_Shockable_Rhythm':'Unknown_AED_Non_Shockable_Rhythm'
    
# }
# # extract concept and calculate similarity
# from ranking_func import rank
# pool = set(['Medical - Abdominal Pain',
#             'Medical - Altered Mental Status',
#             'Medical - Seizure',
#             'Medical - Respiratory Distress/Asthma/COPD/Croup/Reactive Airway',
#             'General - Behavioral/Patient Restraint',
#             'Medical - Overdose/Poisoning - Opioid',
#             'Medical - Diabetic - Hypoglycemia',
#             'Medical - Chest Pain - Cardiac Suspected'])

# def pre_tick_handler(behaviour_tree):
#     blackboard = Blackboard()
#     blackboard.tick_num += 1
    
# pt = 0


# #for i,item in enumerate(tqdm(train_narratives)): *** Update jupyter notebook ***
# for i,item in enumerate((train_narratives)):
#     if (True) :
#         if not pd.isnull(vitals[i]):
#             vt = vitals[i].strip('{}').split('}{')
#             vt = [it.split(':')[-1] for it in vt]
#             for idx,it in enumerate(vt):
#                 if 'EKG-' in it:
#                     temp = it.split('EKG-')
#                     temp[1] = temp[1].replace(' ','_')
#                     temp[1] = temp[1].replace('-','_')
#                     if ',' in temp[1]:
#                         t = temp[1].split(',')
#                         t = [EKGdic[i] for i in t]
#                         vt[idx] = temp[0] + 'EKG-' + ','.join(t)
#                     else:
#                         vt[idx] = temp[0] + 'EKG-' + EKGdic[temp[1]]
#                     if len(temp[1]) > 0:
#                         EKGset.add(temp[1])
#             vt = [ite for l in vt for ite in l.strip().split(' ')]
#             for idx in xrange(len(vt)):
#                 if idx < len(vt) and '-' not in vt[idx]:
#                     vt.pop(idx)
#             for idx,it in enumerate(vt):
#                 temp = it.split('-')
#                 vt[idx] = (temp[0],temp[1])
#     blackboard = Blackboard()
#     blackboard.text = [item]
#     root = py_trees.composites.Sequence("Root_1")
#     IG = be.InformationGathering(inC = vt)
#     TC = be.TextCollection()
#     V = be.Vectorize()
#     root.add_children([TC,IG,V])
#     behaviour_tree = py_trees.trees.BehaviourTree(root)
#     behaviour_tree.add_pre_tick_handler(pre_tick_handler)
#     behaviour_tree.setup(15)
#     behaviour_tree.tick_tock(
#             sleep_ms=50,
#             number_of_iterations=1,
#             pre_tick_handler=None,
#        post_tick_handler=None
#     )
#     pt = i
#     train_vec.append(blackboard.TV)
#     with open('train_vec.txt', 'w') as f:
#         pickle.dump(blackboard.TV, f)
    

In [ ]:
print("hello")

In [ ]:
# with open('train_vec_list.txr','w') as fo:
#     pickle.dump(train_vec, fo)

In [ ]:
# with open('train_vec_list.txr') as fo:
#     train_vec = pickle.load(fo)

In [ ]:
docu = './RAA_1000_test.xlsx'
df = pd.read_excel(docu)
vitals = df['Vitals']
test_narratives = df['Narrative']

In [ ]:
# from tqdm import tqdm_notebook as tqdm
# EKGset = set()
# test_vec = []
# EKGdic = {
#      '':'',
#      'AV_Block_1st_Deg':'AV_Block-1st_Degree',
#      'AV_Block_1st_Degree':'AV_Block-1st_Degree',
#      'AV_Block_2nd_Degree_Type_1':'AV_Block_2nd_Degree_Type_1',
#      'AV_Block_2nd_Degree_Type_2':'AV_Block_2nd_Degree_Type_2',
#      'AV_Block_3rd_Degree':'AV_Block_3rd_Degree',
#      'Asystole':'Asystole',
#      'Artifact':'Artifact',
#      'Atrial_Fibrill':'Atrial_Fibrillation',
#      'Atrial_Fibrillation':'Atrial_Fibrillation',
#      'Atrial_Flutter':'Atrial_Flutter',
#      'Agonal/Idioventricular':'Agonal/Idioventricular',
#      'Juncti':'Junctional_Rhythm',
#      'Junctiona':'Junctional_Rhythm',
#      'Junctional':'Junctional_Rhythm',
#      'Non_STEMI_Anterior_Ischemia':'Non_STEMI_Anterior_Ischemia',
#      'Non_STEMI_Lateral_Ischemia':'Non_STEMI_Lateral_Ischemia',
#      'Other_(Not_Listed)':'Other_(Not_Listed)',
#      'P': 'Paced_Rhythm',
#      'PEA':'Pulseless_Electrical_Activity',
#      'Pac':'Paced_Rhythm',
#      'Paced':'Paced_Rhythm',
#      'Paced_Rhythm':'Paced_Rhythm',
#      'Premature_Ventricular_Contractions':'Premature_Ventricular_Contractions',
#      'Premature_Atrial_Contractions':'Premature_Atrial_Contractions',
#      'Right_Bundle_Branch_Block':'Right_Bundle_Branch_Block',
#      'Left_Bundle_Branch_Block':'Left_Bundle_Branch_Block',
#      'STEMI_Anterior_Ischemia':'STEMI_Anterior_Ischemia',
#      'STEMI_Lateral_Ischemia':'STEMI_Lateral_Ischemia',
#      'STEMI_Inferior_Ischemia':'STEMI_Inferior_Ischemia',
#      'S':'Sinus_Rhythm',
#      'Si':'Sinus_Rhythm',
#      'Sin':'Sinus_Rhythm',
#      'Sinu':'Sinus_Rhythm',
#      'Sinus':'Sinus_Rhythm',
#      'Sinus_':'Sinus_Rhythm',
#      'Sinus_Arrhythmia':'Sinus_Arrhythmia',
#      'Sinus_Bradycardia':'Sinus_Bradycardia',
#      'Sinus_R':'Sinus_Rhythm',
#      'Sinus_Rh':'Sinus_Rhythm',
#      'Sinus_Rhy':'Sinus_Rhythm',
#      'Sinus_Rhyt':'Sinus_Rhythm',
#      'Sinus_Rhyth':'Sinus_Rhythm',
#      'Sinus_Rhythm':'Sinus_Rhythm',
#      'Sinus_Rhythm,Sinus_Tachycardia':'Sinus_Rhythm,Sinus_Tachycardia',
#      'Sinus_T':'Sinus_Tachycardia',
#      'Sinus_Tach':'Sinus_Tachycardia',
#      'Sinus_Tachyc':'Sinus_Tachycardia',
#      'Sinus_Tachycardi':'Sinus_Tachycardia',
#      'Sinus_Tachycardia':'Sinus_Tachycardia',
#      'Supravent':'Supraventricular_Tachycardia',
#      'Supraventricular_Tachycardia':'Supraventricular_Tachycardia',
#      'Torsades_De_Points':'Torsades_De_Points',
#      'Ventricular_Fibrillation':'Ventricular_Fibrillation',
#      'Ventricular_Tachycardia_(With_Pulse)':'Ventricular_Tachycardia',
#      'Ventricular_Tachycardia_(Pulseless)':'Ventricular_Tachycardia_(Pulseless)',
#      'Unknown_AED_Non_Shockable_Rhythm':'Unknown_AED_Non_Shockable_Rhythm'
    
# }
# # extract concept and calculate similarity
# from ranking_func import rank
# pool = set(['Medical - Abdominal Pain',
#             'Medical - Altered Mental Status',
#             'Medical - Seizure',
#             'Medical - Respiratory Distress/Asthma/COPD/Croup/Reactive Airway',
#             'General - Behavioral/Patient Restraint',
#             'Medical - Overdose/Poisoning - Opioid',
#             'Medical - Diabetic - Hypoglycemia',
#             'Medical - Chest Pain - Cardiac Suspected'])

# def pre_tick_handler(behaviour_tree):
#     blackboard = Blackboard()
#     blackboard.tick_num += 1
    
# pt = 0

# #for i,item in enumerate(tqdm(train_narratives)): *** Update jupyter notebook ***
# for i,item in enumerate((test_narratives[1:100])):
#     if not pd.isnull(vitals[i]):
#         vt = vitals[i].strip('{}').split('}{')
#         vt = [it.split(':')[-1] for it in vt]
#         for idx,it in enumerate(vt):
#             if 'EKG-' in it:
#                 temp = it.split('EKG-')
#                 temp[1] = temp[1].replace(' ','_')
#                 temp[1] = temp[1].replace('-','_')
#                 if ',' in temp[1]:
#                     t = temp[1].split(',')
#                     t = [EKGdic[i] for i in t]
#                     vt[idx] = temp[0] + 'EKG-' + ','.join(t)
#                 else:
#                     vt[idx] = temp[0] + 'EKG-' + EKGdic[temp[1]]
#                 if len(temp[1]) > 0:
#                     EKGset.add(temp[1])
#         vt = [ite for l in vt for ite in l.strip().split(' ')]
#         for idx in xrange(len(vt)):
#             if idx < len(vt) and '-' not in vt[idx]:
#                 vt.pop(idx)
#         for idx,it in enumerate(vt):
#             temp = it.split('-')
#             vt[idx] = (temp[0],temp[1])
#     print('VT length',len(vt))
#     blackboard = Blackboard()
#     blackboard.text = [item]
#     root = py_trees.composites.Sequence("Root_1")
#     #IG = be.InformationGathering(inC = vt)
#     IG = be.InformationGathering()
#     TC = be.TextCollection()
#     V = be.Vectorize()
#     root.add_children([TC,IG,V])
#     behaviour_tree = py_trees.trees.BehaviourTree(root)
#     behaviour_tree.add_pre_tick_handler(pre_tick_handler)
#     behaviour_tree.setup(15)
# #     behaviour_tree.tick_tock(
# #             sleep_ms=50,
# #             number_of_iterations=1,
# #             pre_tick_handler=None,
# #        post_tick_handler=None
# #     )
#     print
#     print('##### Item', i)
#     print('S&S:')
#     #print("IG: ", IG.sce.Status)
# #     for i in blackboard.Inters:
# #             print("BB S&S: ", i)
#     #print("IG: ", IG.sce.Status)
#     #print("SCE.concepts: ", IG.sce.concepts) #Empty []
#     #print("VCE.concepts: ", IG.vce.concepts) #Empty []
#     #IG.sce.DisplayStatus()
#     #print('Vital:')
# #     IG.vce.DisplayStatus()
# #     print("IG VCE status: ", IG.vce.Status)
#     #print('Inters:')
#     #IG.ice.DisplayStatus()
#     #print("IG SCE Status: ", IG.sce.Status)
#     #print("TC: ", TC)
#     #print("V: ", V)
#     #print("blackboard.TV", blackboard.TV)
#     pt = i
#     test_vec.append(blackboard.TV)
#     #with open('test_vec.txt', 'w') as f:
#        # pickle.dump(blackboard.TV, f)#

In [ ]:
from tqdm import tqdm_notebook as tqdm
EKGset = set()
test_vec = []
EKGdic = {
     '':'',
     'AV_Block_1st_Deg':'AV_Block-1st_Degree',
     'AV_Block_1st_Degree':'AV_Block-1st_Degree',
     'AV_Block_2nd_Degree_Type_1':'AV_Block_2nd_Degree_Type_1',
     'AV_Block_2nd_Degree_Type_2':'AV_Block_2nd_Degree_Type_2',
     'AV_Block_3rd_Degree':'AV_Block_3rd_Degree',
     'Asystole':'Asystole',
     'Artifact':'Artifact',
     'Atrial_Fibrill':'Atrial_Fibrillation',
     'Atrial_Fibrillation':'Atrial_Fibrillation',
     'Atrial_Flutter':'Atrial_Flutter',
     'Agonal/Idioventricular':'Agonal/Idioventricular',
     'Juncti':'Junctional_Rhythm',
     'Junctiona':'Junctional_Rhythm',
     'Junctional':'Junctional_Rhythm',
     'Non_STEMI_Anterior_Ischemia':'Non_STEMI_Anterior_Ischemia',
     'Non_STEMI_Lateral_Ischemia':'Non_STEMI_Lateral_Ischemia',
     'Other_(Not_Listed)':'Other_(Not_Listed)',
     'P': 'Paced_Rhythm',
     'PEA':'Pulseless_Electrical_Activity',
     'Pac':'Paced_Rhythm',
     'Paced':'Paced_Rhythm',
     'Paced_Rhythm':'Paced_Rhythm',
     'Premature_Ventricular_Contractions':'Premature_Ventricular_Contractions',
     'Premature_Atrial_Contractions':'Premature_Atrial_Contractions',
     'Right_Bundle_Branch_Block':'Right_Bundle_Branch_Block',
     'Left_Bundle_Branch_Block':'Left_Bundle_Branch_Block',
     'STEMI_Anterior_Ischemia':'STEMI_Anterior_Ischemia',
     'STEMI_Lateral_Ischemia':'STEMI_Lateral_Ischemia',
     'STEMI_Inferior_Ischemia':'STEMI_Inferior_Ischemia',
     'S':'Sinus_Rhythm',
     'Si':'Sinus_Rhythm',
     'Sin':'Sinus_Rhythm',
     'Sinu':'Sinus_Rhythm',
     'Sinus':'Sinus_Rhythm',
     'Sinus_':'Sinus_Rhythm',
     'Sinus_Arrhythmia':'Sinus_Arrhythmia',
     'Sinus_Bradycardia':'Sinus_Bradycardia',
     'Sinus_R':'Sinus_Rhythm',
     'Sinus_Rh':'Sinus_Rhythm',
     'Sinus_Rhy':'Sinus_Rhythm',
     'Sinus_Rhyt':'Sinus_Rhythm',
     'Sinus_Rhyth':'Sinus_Rhythm',
     'Sinus_Rhythm':'Sinus_Rhythm',
     'Sinus_Rhythm,Sinus_Tachycardia':'Sinus_Rhythm,Sinus_Tachycardia',
     'Sinus_T':'Sinus_Tachycardia',
     'Sinus_Tach':'Sinus_Tachycardia',
     'Sinus_Tachyc':'Sinus_Tachycardia',
     'Sinus_Tachycardi':'Sinus_Tachycardia',
     'Sinus_Tachycardia':'Sinus_Tachycardia',
     'Supravent':'Supraventricular_Tachycardia',
     'Supraventricular_Tachycardia':'Supraventricular_Tachycardia',
     'Torsades_De_Points':'Torsades_De_Points',
     'Ventricular_Fibrillation':'Ventricular_Fibrillation',
     'Ventricular_Tachycardia_(With_Pulse)':'Ventricular_Tachycardia',
     'Ventricular_Tachycardia_(Pulseless)':'Ventricular_Tachycardia_(Pulseless)',
     'Unknown_AED_Non_Shockable_Rhythm':'Unknown_AED_Non_Shockable_Rhythm'
    
}
# extract concept and calculate similarity
from ranking_func import rank
pool = set(['Medical - Abdominal Pain',
            'Medical - Altered Mental Status',
            'Medical - Seizure',
            'Medical - Respiratory Distress/Asthma/COPD/Croup/Reactive Airway',
            'General - Behavioral/Patient Restraint',
            'Medical - Overdose/Poisoning - Opioid',
            'Medical - Diabetic - Hypoglycemia',
            'Medical - Chest Pain - Cardiac Suspected'])

def pre_tick_handler(behaviour_tree):
    blackboard = Blackboard()
    blackboard.tick_num += 1
    
pt = 0

#for i,item in enumerate(tqdm(train_narratives)): *** Update jupyter notebook ***
for i,item in enumerate((test_narratives[1:100])):
    if not pd.isnull(vitals[i]):
        vt = vitals[i].strip('{}').split('}{')
        vt = [it.split(':')[-1] for it in vt]
        for idx,it in enumerate(vt):
            if 'EKG-' in it:
                temp = it.split('EKG-')
                temp[1] = temp[1].replace(' ','_')
                temp[1] = temp[1].replace('-','_')
                if ',' in temp[1]:
                    t = temp[1].split(',')
                    t = [EKGdic[i] for i in t]
                    vt[idx] = temp[0] + 'EKG-' + ','.join(t)
                else:
                    vt[idx] = temp[0] + 'EKG-' + EKGdic[temp[1]]
                if len(temp[1]) > 0:
                    EKGset.add(temp[1])
        vt = [ite for l in vt for ite in l.strip().split(' ')]
        for idx in xrange(len(vt)):
            if idx < len(vt) and '-' not in vt[idx]:
                vt.pop(idx)
        for idx,it in enumerate(vt):
            temp = it.split('-')
            vt[idx] = (temp[0],temp[1])
    blackboard = Blackboard()
    blackboard.text = [item]
    root = py_trees.composites.Sequence("Root_1")
#     IG = be.InformationGathering(inC = vt)
    IG = be.InformationGathering(item)
    #IG.sce.concepts
    #be.IG.update()
    TC = be.TextCollection()
    V = be.Vectorize()
    root.add_children([TC,IG,V])
    behaviour_tree = py_trees.trees.BehaviourTree(root)
    behaviour_tree.add_pre_tick_handler(pre_tick_handler)
    behaviour_tree.setup(15)
#     behaviour_tree.tick_tock(
#             sleep_ms=50,
#             number_of_iterations=1,
#             pre_tick_handler=None,
#        post_tick_handler=None
#     )
    print
    print('##### Item', item)
    print('S&S:')
    print('IG.slist', IG.slist)
    print('IG.sce', IG.sce)
    #print('IG.sce.Status', IG.sce.Status)
    #print('IG.sce',) 
    IG.setup()
    IG.Vital2Symptom()
    IG.update()
    print('IG.sce', IG.sce)
    
#    print("IG: ", IG.sce.Status)
#     for i in blackboard.Inters:
#             print("BB S&S: ", i)
    #print("IG: ", IG.sce.Status)
#     print("SCE.concepts: ", IG.sce.concepts) #Empty []
#     print("VCE.concepts: ", IG.vce.concepts) #Empty []
#    IG.sce.DisplayStatus()
    #print('Vital:')
#     IG.vce.DisplayStatus()
#    print("IG VCE status: ", IG.vce.Status)
    #print('Inters:')
    #IG.ice.DisplayStatus()
    #print("IG SCE Status: ", IG.sce.Status)
    #print("TC: ", TC)
    #print("V: ", V)
    #print("blackboard.TV", blackboard.TV)
    pt = i
    #test_vec.append(blackboard.TV)
    #with open('test_vec.txt', 'w') as f:
       # pickle.dump(blackboard.TV, f)#

In [ ]:
print("There")

In [55]:
import ConceptExtract as CE
ObjConcept = CE.ConceptExtractor("concept_list(s&s)_revised.csv")
ObjConcept.StatusInit()
ObjConcept.ConceptExtract(test_narratives[1])
ObjConcept.concepts
print(ObjConcept.concepts)
ObjConcept.FirstExtract(test_narratives[1],1)
ObjConcept.concepts
print(ObjConcept.concepts)
#CE.CEWithoutMM("/renewconceptlists/All_In_One_signs&symptoms.xlsx","","")

Check initial
{'epinephrine': <ConceptExtract.PatientStatus object at 0x7f3dcb7cebd0>, 'bizarre behavior': <ConceptExtract.PatientStatus object at 0x7f3e219c7790>, 'violent': <ConceptExtract.PatientStatus object at 0x7f3dcb7ce950>, 'nitroglycerin': <ConceptExtract.PatientStatus object at 0x7f3e219c7750>, 'rigidity': <ConceptExtract.PatientStatus object at 0x7f3dcb7ce090>, 'rhonchi': <ConceptExtract.PatientStatus object at 0x7f3dcb7cee50>, 'fentanyl': <ConceptExtract.PatientStatus object at 0x7f3e219c7a90>, 'midazolam': <ConceptExtract.PatientStatus object at 0x7f3e219c77d0>, 'rash': <ConceptExtract.PatientStatus object at 0x7f3dcb7cea90>, 'substance abuse history': <ConceptExtract.PatientStatus object at 0x7f3dcb145050>, 'absent femoral pulses': <ConceptExtract.PatientStatus object at 0x7f3e219c7d50>, 'cool skin': <ConceptExtract.PatientStatus object at 0x7f3e219c79d0>, 'use of accessory muscles': <ConceptExtract.PatientStatus object at 0x7f3e219c7650>, 'lightheadedness': <ConceptExtra

In [23]:
with open('test_vec_list.txr','w') as fo:
    pickle.dump(test_vec, fo)

In [24]:
with open('test_vec_list.txr') as fo:
    test_vec = pickle.load(fo)

In [25]:
len(test_vec)

1000

In [ ]:
#train_vec and test_vec hold the signs and symptoms extracted using ConceptExtract.py and behvaiours

#-what are test and train_vec exactly???

In [ ]:
#Need to also include extraction using newest code from github
# ie ConceptExtract.py ... behvaiours_m.py and all dependencies

In [ ]:
# Then do S&S concept extract using w and w/o metamap

In [33]:
help(Blackboard)


SyntaxError: invalid syntax (<ipython-input-33-d178aa4ecb66>, line 1)

In [35]:
narra,intdict = load_RAA_data('./RAA_train.xlsx')
test_narra, _ = load_RAA_data('./RAA_1000_test.xlsx', cv = False)
risk_route = './Intervention Safety Sheet.xlsx'
df_risk = pd.read_excel(risk_route)
int2fn_score = dict()
int2fp_score = dict()
for row in df_risk.iterrows():
    name = row[1]['Intervention'].split('\'')[1]
    FN_score, FP_score = 0, 0
    if not pd.isnull(row[1]['If NOT Done When Indicated']):
        FN_score = int(row[1]['If NOT Done When Indicated'])
    if not pd.isnull(row[1]['If Done When NOT Indicated']):
        FP_score = int(row[1]['If Done When NOT Indicated'])
    if not FN_score or not FP_score or (name not in intdict):
        continue
    int2fn_score[name] = FN_score
    int2fp_score[name] = FP_score
int2num = dict()
num2int = dict()
for i,key in enumerate(int2fn_score):
    int2num[key] = i
    num2int[i] = key
# n = NLP()
# load technical n-grams
# fo = open('ngrams.txt')
# ngrams = set()
# for line in fo:
#     if line == '\n': continue
#     ngrams.add(line.strip('\n'))
# fo.close()

In [36]:
inter_safety = [dict() for _ in range(len(int2num))]
for idx in range(len(num2int)):
    inter_safety[idx][0] = 1. / int2fn_score[num2int[idx]]
    inter_safety[idx][1] = 1. / int2fp_score[num2int[idx]]
inter_safety_dic = dict()
for idx in range(len(num2int)):
    inter_safety_dic[idx] = int2fp_score[num2int[idx]]

In [37]:
total_text = [i.text for i in narra]
total_inter = [i.inter for i in narra]

In [38]:
train,_ = train_test_split(narra, random_state=46, test_size=.2, shuffle=True)
train_text = [i.text for i in train]
train_inter = [i.inter for i in train]
test_text = [i.text for i in test_narra]
test_inter = [i.inter for i in test_narra]

In [39]:
def preprocess(text):
    text = text.lower()
    text = cleanPunc(text)
    text = keepAlpha(text)
    text = stemming(text)
    return text

#

vectorizer = TfidfVectorizer(ngram_range=(1,1), preprocessor = preprocess, stop_words = 'english', norm='l2')
vectorizer.fit(train_text)
x_train = vectorizer.transform(train_text)
y_train = [[int2num[inter] for inter in case if inter in int2num] for case in train_inter]
encoded_y_train = np.array([[int(num in case) for num in range(len(int2num))] for case in y_train])
x_test = vectorizer.transform(test_text)
y_test = [[int2num[inter] for inter in case if inter in int2num] for case in test_inter]
encoded_y_test = np.array([[int(num in case) for num in range(len(int2num))] for case in y_test])

/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens [u'abov', u'afterward', u'alon', u'alreadi', u'alway', u'ani', u'anoth', u'anyon', u'anyth', u'anywher', u'becam', u'becaus', u'becom', u'befor', u'besid', u'cri', u'describ', u'dure', u'els', u'elsewher', u'empti', u'everi', u'everyon', u'everyth', u'everywher', u'fifti', u'forti', u'henc', u'hereaft', u'herebi', u'howev', u'hundr', u'inde', u'mani', u'meanwhil', u'moreov', u'nobodi', u'noon', u'noth', u'nowher', u'onc', u'onli', u'otherwis', u'ourselv', u'perhap', u'pleas', u'sever', u'sinc', u'sincer', u'sixti', u'someon', u'someth', u'sometim', u'somewher', u'themselv', u'thenc', u'thereaft', u'therebi', u'therefor', u'togeth', u'twelv', u'twenti', u'veri', u'whatev', u'whenc', u'whenev', u'wherea', u'whereaft', u'wherebi', u'wherev', u'whi', u'yourselv'] not in stop

In [40]:
vectorizer = TfidfVectorizer(ngram_range=(1,1), preprocessor = preprocess, stop_words = 'english', norm='l2')
vectorizer.fit(total_text)
x_total = vectorizer.transform(total_text)
y_total = [[int2num[inter] for inter in case if inter in int2num] for case in total_inter]
encoded_y_total = np.array([[int(num in case) for num in range(len(int2num))] for case in y_total])
class_weight = np.sum(encoded_y_total, axis = 0)

In [78]:
scoring = {'precision_weighted': make_scorer(weighted_precision, weight = class_weight),
           'recall_weighted': make_scorer(weighted_recall, weight = class_weight),
           'f1_weighted': make_scorer(weighted_f1, weight = class_weight),
           'precision_micro': 'precision_micro',
           'recall_micro': 'recall_micro',
           'f1_micro': 'f1_micro'}

In [79]:
scores_table = []

In [80]:
# clfs using feature vectors
clf_16 = OneVsRestClassifier(SVC(kernel = 'linear', probability=True))
scores_16 = cross_validate(clf_16, train_vec, encoded_y_total, scoring=scoring,
                         cv=5, n_jobs=-1, return_train_score=False, return_estimator=True)
show_results(scores_16)

KeyboardInterrupt: 

In [ ]:
clf_16 = OneVsRestClassifier(SVC(kernel = 'linear', probability=True))
clf_16.fit(train_vec, encoded_y_total)
y_pre_16 = clf_16.predict(test_vec)
y_pos_16 = clf_16.predict_proba(test_vec)
show_test_results(encoded_y_test, y_pre_16, y_pos_16, class_weight)

In [81]:
clf_rf_6 = RandomForestClassifier()
scores_17 = cross_validate(clf_rf_6, train_vec, encoded_y_total, scoring=scoring,
                         cv=5, n_jobs=-1, return_train_score=False, return_estimator=True)
show_results(scores_17)

test_precision_weighted:0.63
test_recall_weighted:0.61
test_f1_weighted:0.58
test_precision_micro:0.79
test_recall_micro:0.60
test_f1_micro:0.68


In [82]:
clf_rf_6 = RandomForestClassifier()
clf_rf_6.fit(train_vec, encoded_y_total)
y_pre_17 = clf_rf_6.predict(test_vec)
y_pos_17 = clf_rf_6.predict_proba(test_vec)
show_test_results(encoded_y_test, y_pre_17, trans_prob(y_pos_17), class_weight)

/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ValueError: Found input variables with inconsistent numbers of samples: [1000, 4]

In [ ]:
clf_dt_6 = DecisionTreeClassifier()
scores_18 = cross_validate(clf_dt_6, train_vec, encoded_y_total, scoring=scoring,
                         cv=5, n_jobs=-1, return_train_score=False, return_estimator=True)
show_results(scores_18)

In [ ]:
clf_dt_6 = DecisionTreeClassifier()
clf_dt_6.fit(train_vec, encoded_y_total)
y_pre_18 = clf_dt_6.predict(test_vec)
y_pos_18 = clf_dt_6.predict_proba(test_vec)
show_test_results(encoded_y_test, y_pre_18, trans_prob(y_pos_18), class_weight)